## API call Functions
This section handles all API calls to octopus

In [1]:

import requests
import json


# Your Octopus Energy API key


# API_KEY = "YOUR_API_KEY"


# Function to fetch electricity consumption data for a specific MPAN and meter serial number


def get_electricity_consumption(mpan, meter_serial_number, period_from, period_to):

    url = f"https://api.octopus.energy/v1/electricity-meter-points/{mpan}/meters/{






        meter_serial_number}/consumption?page_size=14000&period_from={period_from}&period_to={period_to}"

    response = requests.get(url, auth=(API_KEY, ''))

    return response.json()


# Function to fetch tariff prices for a specific product and tariff code


def get_tariff_prices_old(tariff_code, period_from, period_to):
    product_code = tariff_code[5:-2]

    url = f"https://api.octopus.energy/v1/products/{product_code}/electricity-tariffs/{






        tariff_code}/standard-unit-rates?page_size=14000&period_from={period_from}&period_to={period_to}"

    # print(url)
    response = requests.get(url, auth=(API_KEY, ''))

    return response.json()


def get_tariff_prices(tariff_code, period_from, period_to):
    product_code = tariff_code[5:-2]
    url = f"https://api.octopus.energy/v1/products/{product_code}/electricity-tariffs/{
        tariff_code}/standard-unit-rates?page_size=14000&period_from={period_from}&period_to={period_to}"
    response = requests.get(url, auth=(API_KEY, ''))
    data = response.json()
    all_data = data['results']

    while data['next'] is not None:
        # Get the next page URL
        next_page_url = data['next']
        response = requests.get(next_page_url, auth=(API_KEY, ''))
        data = response.json()
        all_data.extend(data['results'])

    return all_data


# Main function to gather consumption data and tariff costs


def get_electricity_data(account_number, start_date, end_date):

    # Get account details to get MPAN and meter information

    account_url = f"https://api.octopus.energy/v1/accounts/{account_number}/"

    account_response = requests.get(account_url, auth=(API_KEY, ''))

    account_data = account_response.json()

    # Extract MPAN and meter information

    mpans = [meter_point['mpan']






             for meter_point in account_data['properties'][0]['electricity_meter_points']]

    meter_serial_numbers = [meter['serial_number']






                            for meter in account_data['properties'][0]['electricity_meter_points'][0]['meters']]
    tariff_code = account_data['properties'][0]['electricity_meter_points'][0]['agreements'][0]['tariff_code']

    # Fetch consumption data for each MPAN and meter

    consumption_data = {}
    mpan_file = open('mpan_meter.csv', 'w')

    for mpan in mpans:

        for meter_serial_number in meter_serial_numbers:

            mpan_file.write(f'{mpan}-{meter_serial_number}\n')

            consumption_data[(mpan, meter_serial_number)] = get_electricity_consumption(






                mpan, meter_serial_number, start_date, end_date)
            with open(f'{mpan}-{meter_serial_number}.json', 'w') as f:
                json.dump(consumption_data[(mpan, meter_serial_number)], f)

    mpan_file.close()

    # Fetch tariff prices for all tariffs associated with the account
    tariff_file = open('tariffs.csv', 'w')

    tariff_codes = [tariff['tariff_code'] for tariff in account_data['properties']






                    [0]['electricity_meter_points'][0]['agreements']]

    tariff_prices = {}

    for tariff_code in tariff_codes:
        tariff_file.write(f'{tariff_code}\n')

        tariff_prices[tariff_code] = get_tariff_prices(






            tariff_code, start_date, end_date)
        with open(f'{tariff_code}.json', 'w') as f:
            json.dump(tariff_prices[tariff_code], f)

    tariff_file.close()

    return {






        "consumption_data": consumption_data,






        "tariff_prices": tariff_prices






    }

## API calling execution
This part is the actual execution to get tarrifs and consumption based on functions above

In [2]:

# Usage example
account_number = "A-3FAEDC7D"
start_date = "2024-04-19T01:00:00Z"
end_date = "2025-01-27T00:00:00Z"

# load keys

# get API key from file
with open('apikey.txt', 'r') as file:
    API_KEY = file.read()

electricity_data = get_electricity_data(account_number, start_date, end_date)

## Download energy selling price in Agile

In [4]:
# Get export cost data
import_data = get_tariff_prices(
    'E-1R-AGILE-BB-24-10-01-H', start_date, end_date)
export_data = get_tariff_prices(
    'E-1R-AGILE-OUTGOING-BB-23-02-28-H', start_date, end_date)
with open('export.json', 'w') as f:
    json.dump(export_data, f)
with open('import.json', 'w') as f:
    json.dump(import_data, f)
    

## Run this section after each time the data gets downloaded to convert Jsons to CSV

In [10]:

import json
import csv
import os


def json_to_csv(json_file_path, csv_file_path):
    """
    Converts a JSON file to a CSV file.

    Args:
        json_file_path: Path to the JSON file.
        csv_file_path: Path to the output CSV file.
    """
    
   
    try:
        with open(json_file_path, 'r') as f:
            data = json.load(f)

        try:
            results = data.get('results', [])  # Extract the 'results' list
        except AttributeError:
            results = data

        if not results:
            print("No 'results' found in the JSON file.")
            return

        # Extract field names from the first dictionary in results
        fieldnames = results[0].keys() if results else []

        with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            writer.writeheader()  # Write the header row
            writer.writerows(results)  # Write the data rows

        print(f"Successfully converted {json_file_path} to {csv_file_path}")
        

    except FileNotFoundError:
        print(f"Error: JSON file not found at {json_file_path}")
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in {json_file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")
        


# Get the current directory
current_dir = os.getcwd()

for filename in os.listdir(current_dir):
    # Check if the file is a JSON file
    if filename.endswith(".json"):

        # Construct the full file path
        json_file_name = filename
        # Define the output CSV file name based on the JSON file name
        csv_file_name = json_file_name.replace(".json", ".csv")
        json_to_csv(json_file_name, csv_file_name)

Successfully converted export.json to export.csv
Successfully converted 2600002565617-22L4294637.json to 2600002565617-22L4294637.csv
Successfully converted E-1R-INTELLI-VAR-22-10-14-H.json to E-1R-INTELLI-VAR-22-10-14-H.csv
Successfully converted import.json to import.csv
Successfully converted E-1R-VAR-22-11-01-H.json to E-1R-VAR-22-11-01-H.csv


# breakpoint All files downloaded now. 
## Rest of this code is junk. Develop code to merge 3 CSV files - consumption, Buy rate, sell rate


In [12]:
# File configuration for lookup
consumption_file = '2600002565617-22L4294637.csv'
buy_rate_file = 'import.csv'
sell_rate_file = 'export.csv'
#test = pd.read_csv(consumption_file)

In [15]:
import pandas as pd


def add_rates_to_consumption_optimized(consumption_file, buy_rate_file, sell_rate_file):
    """
    Adds buy and sell rate columns to consumption data, with debugging and a fallback.
    """
    try:
        # Read data (optimized as before)
        consumption_df = pd.read_csv(
            consumption_file,
            parse_dates=["interval_start", "interval_end"],
            dtype={"consumption": "float32"},
        )
        buy_rate_df = pd.read_csv(
            buy_rate_file,
            parse_dates=["valid_from", "valid_to"],
            dtype={"value_inc_vat": "float32"},
        )
        sell_rate_df = pd.read_csv(
            sell_rate_file,
            parse_dates=["valid_from", "valid_to"],
            dtype={"value_inc_vat": "float32"},
        )
    except FileNotFoundError:
        print("Error: One or more input files not found.")
        return None

    # Prepare dataframes for merge_asof
    consumption_df = consumption_df.sort_values("interval_start")
    buy_rate_df = buy_rate_df.sort_values("valid_from")
    sell_rate_df = sell_rate_df.sort_values("valid_from")

    # Debugging: Check for duplicates again (just to be absolutely sure)
    print("Buy rate duplicates:", buy_rate_df["valid_from"].duplicated().sum())
    print("Sell rate duplicates:",
          sell_rate_df["valid_from"].duplicated().sum())

    # Debugging: Re-confirm data types
    print("Consumption interval_start type:",
          consumption_df["interval_start"].dtype)
    print("Buy rate valid_from type:", buy_rate_df["valid_from"].dtype)
    print("Sell rate valid_from type:", sell_rate_df["valid_from"].dtype)

    try:
        # Perform merge_asof (with error handling)
        merged_df = pd.merge_asof(
            consumption_df,
            buy_rate_df[["valid_from", "value_inc_vat"]].rename(
                columns={"value_inc_vat": "buy_rate_inc_vat"}
            ),
            left_on="interval_start",
            right_on="valid_from",
            direction="backward",
        )

        merged_df = pd.merge_asof(
            merged_df,
            sell_rate_df[["valid_from", "value_inc_vat"]].rename(
                columns={"value_inc_vat": "sell_rate_inc_vat"}
            ),
            left_on="interval_start",
            right_on="valid_from",
            direction="backward",
        )

        merged_df = merged_df.drop(
            columns=["valid_from_x", "valid_from_y"], errors="ignore"
        )
        return merged_df

    except Exception as e:
        print(f"Error during merge_asof: {e}")
        print("Falling back to less efficient method...")

        # Fallback: Iterative method (less efficient but should always work)
        # (Code from your original, corrected iterative approach)
        results = []
        for index, row in consumption_df.iterrows():
            interval_start = row["interval_start"]
            interval_end = row["interval_end"]

            buy_rate = buy_rate_df.loc[
                (buy_rate_df["valid_from"] < interval_end)
                & (buy_rate_df["valid_to"] > interval_start),
                "value_inc_vat",
            ]

            sell_rate = sell_rate_df.loc[
                (sell_rate_df["valid_from"] < interval_end)
                & (sell_rate_df["valid_to"] > interval_start),
                "value_inc_vat",
            ]

            buy_rate_value = buy_rate.iloc[0] if not buy_rate.empty else None
            sell_rate_value = sell_rate.iloc[0] if not sell_rate.empty else None

            results.append(
                {
                    "consumption": row["consumption"],
                    "interval_start": interval_start,
                    "interval_end": interval_end,
                    "buy_rate_inc_vat": buy_rate_value,
                    "sell_rate_inc_vat": sell_rate_value,
                }
            )

        result_df = pd.DataFrame(results)
        return result_df


result_df = add_rates_to_consumption_optimized(
    consumption_file, buy_rate_file, sell_rate_file
)

if result_df is not None:
    print(result_df)
    result_df.to_csv("output.csv", index=False)

Buy rate duplicates: 0
Sell rate duplicates: 0
Consumption interval_start type: object
Buy rate valid_from type: datetime64[ns, UTC]
Sell rate valid_from type: datetime64[ns, UTC]
Error during merge_asof: Incompatible merge dtype, datetime64[ns, UTC] and dtype('O'), both sides must have numeric dtype
Falling back to less efficient method...
       consumption             interval_start               interval_end  \
0            0.065  2024-04-19 02:00:00+01:00  2024-04-19 02:30:00+01:00   
1            0.072  2024-04-19 02:30:00+01:00  2024-04-19 03:00:00+01:00   
2            0.097  2024-04-19 03:00:00+01:00  2024-04-19 03:30:00+01:00   
3            0.074  2024-04-19 03:30:00+01:00  2024-04-19 04:00:00+01:00   
4            0.066  2024-04-19 04:00:00+01:00  2024-04-19 04:30:00+01:00   
...            ...                        ...                        ...   
13578        0.426  2025-01-26 22:00:00+00:00  2025-01-26 22:30:00+00:00   
13579        0.838  2025-01-26 22:30:00+00:00  20